In [1]:
import numpy
import pandas as pd
import qgrid
import datetime
import calendar

In [2]:
data1=pd.read_csv('crashes.csv',low_memory=False)

In [3]:
datap1=pd.read_csv('persons.csv',low_memory=False)

In [4]:
dataw1=pd.read_csv('weather.csv',low_memory=False)

In [5]:
# Crashes
print("Initial Size:",data1.shape[0])
data1=data1.dropna(subset=['LATITUDE'])
print("Mod Size:",data1.shape[0])
data1=data1.dropna(subset=['LONGITUDE'])
print("Mod Size:",data1.shape[0])
data1= data1[data1['LATITUDE'] != 0]
data1= data1[data1['LONGITUDE'] != 0]
print("Mod Size:",data1.shape[0])
data1=data1[data1['CONTRIBUTING FACTOR VEHICLE 1'].str.contains("Unspecified")==False]
print("Mod Size:",data1.shape[0])
data1=data1.drop(columns=['LOCATION'])
data1=data1.drop(columns=['ZIP CODE','BOROUGH'])
data1['KILLED/INJURED']=data1['NUMBER OF PERSONS KILLED'] + data1['NUMBER OF PERSONS INJURED']
print("Mod Size:",data1.shape[0])
data1=data1.drop(columns=['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',])
data1=data1.dropna(subset=['VEHICLE TYPE CODE 1'])
print("Mod Size:",data1.shape[0])
data1['CRASH TIME']=data1['CRASH TIME'].str.replace(r'[:].*','')

def findDay(date):
    born = datetime.datetime.strptime(date, '%m/%d/%Y').weekday()
    return (calendar.day_name[born])
data1['WEEKDAY']=data1['CRASH DATE'].apply(lambda x:findDay(x))
data1['CRASH DATE']=pd.to_datetime(data1['CRASH DATE'])
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal = calendar()
holidays = cal.holidays(start=data1['CRASH DATE'].min(), end=data1['CRASH DATE'].max())
data1['Holiday']=data1['CRASH DATE'].isin(holidays)

Initial Size: 1877557
Mod Size: 1659479
Mod Size: 1659479
Mod Size: 1657071
Mod Size: 1073637
Mod Size: 1073637
Mod Size: 1068838


/tmp/ipykernel_481818/258530685.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  data1['CRASH TIME']=data1['CRASH TIME'].str.replace(r'[:].*','')


In [6]:
# Person Crashes
datap1=datap1.drop(columns=['UNIQUE_ID','PERSON_TYPE','PERSON_INJURY',
                            'EJECTION','EMOTIONAL_STATUS','BODILY_INJURY','POSITION_IN_VEHICLE',
                            'SAFETY_EQUIPMENT','PED_LOCATION','PED_ACTION','COMPLAINT','PED_ROLE',
                            'CONTRIBUTING_FACTOR_1','CONTRIBUTING_FACTOR_2'])
print("Mod Size:",datap1.shape[0])
datap1=datap1.dropna(subset=['PERSON_SEX'])
print("Mod Size:",datap1.shape[0])
datap1=datap1[datap1['PERSON_SEX'].str.contains("U")==False]
print("Mod Size:",datap1.shape[0])
datap1=datap1.dropna(subset=['PERSON_AGE'])
datap1 = datap1[~(datap1['PERSON_AGE'] < 16)]
print("Mod Size:",datap1.shape[0])
datap1=datap1.dropna(subset=['PERSON_ID'])
print("Mod Size:",datap1.shape[0])
datap1=datap1.dropna(subset=['VEHICLE_ID'])
print("Mod Size:",datap1.shape[0])
datap1['CRASH_TIME']=datap1['CRASH_TIME'].str.replace(r'[:].*','')
datap1['CRASH_DATE']=pd.to_datetime(datap1['CRASH_DATE'])

Mod Size: 4626711
Mod Size: 4093419
Mod Size: 3701767
Mod Size: 3281713
Mod Size: 3281700
Mod Size: 3223619


/tmp/ipykernel_481818/3516917435.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  datap1['CRASH_TIME']=datap1['CRASH_TIME'].str.replace(r'[:].*','')


In [7]:
# Weather Crashes
dataw1=dataw1.drop(columns=['dt','timezone','city_name','lat','lon'])
dataw1['dt_iso']=dataw1['dt_iso'].str.replace(r'[+].*','')
dataw1['dt_iso']=pd.to_datetime(dataw1['dt_iso'])
dataw1['hour']=dataw1['dt_iso'].apply(lambda x:x.hour)
dataw1['dt_iso']=pd.to_datetime(dataw1['dt_iso']).dt.date
dataw1=dataw1.rename(columns={'dt_iso':'CRASH_DATE'})
dataw1=dataw1.rename(columns={'hour':'CRASH_TIME'})
print(dataw1.columns)
print("Mod Size:",dataw1.shape[0])

/tmp/ipykernel_481818/1256670018.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  dataw1['dt_iso']=dataw1['dt_iso'].str.replace(r'[+].*','')


Index(['CRASH_DATE', 'temp', 'visibility', 'dew_point', 'temp_min', 'temp_max',
       'pressure', 'sea_level', 'grnd_level', 'humidity', 'wind_speed',
       'wind_deg', 'wind_gust', 'rain_1h', 'rain_3h', 'snow_1h', 'snow_3h',
       'clouds_all', 'weather_id', 'weather_main', 'weather_description',
       'weather_icon', 'CRASH_TIME'],
      dtype='object')
Mod Size: 95994


In [8]:
# Merging
print("Data 1:",data1.shape[0])
print("Datap1:",datap1.shape[0])
print("Dataw1:",dataw1.shape[0])

Data 1: 1068838
Datap1: 3223619
Dataw1: 95994


In [9]:
# Merging Crashes and person
datam1=datap1.merge(data1,on='COLLISION_ID')
print("Datam1:",datam1.shape[0])
datam1=datam1.drop(columns=['PERSON_ID','CRASH DATE','CRASH TIME','VEHICLE TYPE CODE 5','VEHICLE TYPE CODE 4',
                            'VEHICLE TYPE CODE 3','CONTRIBUTING FACTOR VEHICLE 5','CONTRIBUTING FACTOR VEHICLE 4',
                           'CONTRIBUTING FACTOR VEHICLE 3'])

Datam1: 2346140


In [20]:
print(dataw1.dtypes)
dataw1['CRASH_DATE'] = dataw1['CRASH_DATE'].astype('datetime64[ns]')
datam1['CRASH_TIME'] = datam1['CRASH_TIME'].astype(int)
print(dataw1.dtypes)
print(datam1.dtypes)


CRASH_DATE             datetime64[ns]
temp                          float64
visibility                    float64
dew_point                     float64
temp_min                      float64
temp_max                      float64
pressure                        int64
sea_level                     float64
grnd_level                    float64
humidity                        int64
wind_speed                    float64
wind_deg                        int64
wind_gust                     float64
rain_1h                       float64
rain_3h                       float64
snow_1h                       float64
snow_3h                       float64
clouds_all                      int64
weather_id                      int64
weather_main                   object
weather_description            object
weather_icon                   object
CRASH_TIME                      int64
dtype: object
CRASH_DATE             datetime64[ns]
temp                          float64
visibility                    float6

In [21]:
## Merge Weather
print("Datam1:",datam1.shape[0])
datam2=datam1.merge(dataw1,on=['CRASH_DATE','CRASH_TIME'])
print("Datam2:",datam2.shape[0])

Datam1: 2346140
Datam2: 2529009


In [25]:
datam2.to_csv('finalData.csv')

In [23]:
widget = qgrid.show_grid(datam1, show_toolbar=True,grid_options={'forceFitColumns': False, 'defaultColumnWidth': 150})
widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [24]:
del widget
widget = qgrid.show_grid(datam2, show_toolbar=True,grid_options={'forceFitColumns': False, 'defaultColumnWidth': 150})
widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…